In [21]:
import pandas as pd
import datetime
import numpy as np
import os

print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Analysis/2020_Q1/plannder_requests/YG_zips_all_current_stores
2020-05-14 17:12:10.504581


In [22]:
df_YG_store_list=pd.read_excel("./OnboardingKit BL 0427.xlsx",dtype=str,sheet_name="List of stores")
df_YG_store_list=df_YG_store_list[['Store']].rename(columns={"Store":"location_id"}).drop_duplicates()
df_YG_store_list.shape

(1426, 1)

In [23]:
excel_file_TA=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
excel_file_TA.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [24]:
df_store_TA=excel_file_TA.parse("view_by_store",dtype=str,usecols=['ta_name','location_id','TA_num'])
df_YG_store_list=pd.merge(df_YG_store_list,df_store_TA,on="location_id",how="left")

,TA_num,ta_name,location_id
0,1,multiple_1,1949
1,1,multiple_1,1255


In [30]:
df_YG_store_list_existing=df_YG_store_list[pd.notnull(df_YG_store_list['TA_num'])]
df_YG_store_list_existing['TA_num']=df_YG_store_list_existing['TA_num'].astype(int)
df_YG_store_list_existing['TA_num'].min(),df_YG_store_list_existing['TA_num'].max(),df_YG_store_list_existing['TA_num'].nunique()

,location_id,TA_num,ta_name
0,1,121,multiple_121
1,3,227,single_3


In [28]:
df_YG_store_list_new=df_YG_store_list[pd.isnull(df_YG_store_list['TA_num'])]
df_YG_store_list_new=df_YG_store_list_new.sort_values("location_id")
df_YG_store_list_new['TA_num']=range(707,707+len(df_YG_store_list_new))
df_YG_store_list_new['ta_name']="new_"+df_YG_store_list_new['location_id']


In [32]:
df_YG_store_list=df_YG_store_list_existing.append(df_YG_store_list_new)
df_YG_store_list['location_id']=df_YG_store_list['location_id'].astype(int)
df_YG_store_list=df_YG_store_list.sort_values("location_id")

In [33]:
df_YG_store_list

,location_id,TA_num,ta_name
0,1,121,multiple_121
1,3,227,single_3
2,29,594,single_29
3,30,235,single_30
4,42,79,multiple_79
...,...,...,...
1421,5420,758,new_5420
1422,5421,759,new_5421
1423,5422,760,new_5422
1424,5423,761,new_5423


In [37]:
df_zips_by_store=pd.read_excel("./BL_TA_zips_JL_2020-04-30.xlsx",sheet_name="store_with_zips",dtype=str)
df_zips_by_store['location_id']=df_zips_by_store['location_id'].astype(int)
df_zips_by_store=pd.merge(df_zips_by_store,df_YG_store_list,on="location_id")

In [44]:
# In case if you want to replace the zip label first
# df_zips_by_store['zip_type']=df_zips_by_store['zip_type'].replace("10_zip_count","P")

df_summary_by_TA=df_zips_by_store.groupby(['TA_num','ta_name','zip_type'])['zip_cd'].nunique().to_frame().reset_index()
df_summary_by_TA=df_summary_by_TA.pivot_table(index=['TA_num','ta_name'],columns="zip_type",values="zip_cd").reset_index()
df_summary_by_TA=df_summary_by_TA.rename(columns={"P":"P_zip_count","S":"S_zip_count","zip_10":"10_zip_count"})

df_summary_by_TA_2=df_zips_by_store.groupby(['TA_num','ta_name'])['location_id'].nunique().to_frame().reset_index().rename(columns={"location_id":"store_count"})
df_summary_by_TA=pd.merge(df_summary_by_TA,df_summary_by_TA_2,on=['TA_num','ta_name'])
df_summary_by_TA=df_summary_by_TA.fillna(0)

In [45]:
writer=pd.ExcelWriter("./BL_additional_2_needs_filed_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_YG_store_list.to_excel(writer,"TA_mapping_to_store",index=False)
df_summary_by_TA.to_excel(writer,"summary_by_TA",index=False)
writer.save()